In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas_profiling as pp
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
# Site URL
url="http://www.maxifoot.fr/classement-buteur-europe.htm"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")

### Scraping des sites de chaque saison

results = soup.find(id='mb6a')

web = []
for i in results.find_all("a") :
    web.append("https://www.maxifoot.fr/" + i['href'])

web

['https://www.maxifoot.fr/classement-buteur-europe.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2020.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2019.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2018.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2017.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2016.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2015.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2014.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2013.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2012.htm',
 'https://www.maxifoot.fr/classement-buteur-europe-2011.htm']

In [3]:
### Tables des années allant de 2021 to 2011

def get_table_of_players(web) :        
        # Site URL
        url = web

        # Make a GET request to fetch the raw HTML content
        html_content = requests.get(url).text

        # Parse HTML code for the entire site
        soup = BeautifulSoup(html_content, "lxml")


        gdp = soup.find_all("table")


        # Lets go ahead and scrape first table with HTML code gdp[0]
        table = gdp[11]
        # the head will form our column names
        body = table.find_all("tr")
        # Head values (Column names) are the first items of the body list
        head = body[0] # 0th item is the header row
        body_rows = body[1:] # All other items becomes the rest of the rows
        body_rows


        # Declare empty list to keep Columns names
        headings = []
        for item in head.find_all("th"): # loop through all th elements
            # convert the th elements to text and strip "\n"
            item = (item.text).rstrip("\n")
            # append the clean column name to headings
            headings.append(item)


        all_rows = [] # will be a list for list for all rows
        for row_num in range(len(body_rows)): # A row at a time
            row = [] # this will old entries for one row
            for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
                # row_item.text removes the tags from the entries
                # the following regex is to remove \xa0 and \n and comma from row_item.text
                # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
                aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
                #append aa to row - note one row entry is being appended
                row.append(aa)
            # append one row to all_rows
            all_rows.append(row)

        df = pd.DataFrame(data=all_rows,columns=headings)
        df = pd.DataFrame(data=all_rows,columns=headings)
        year = web.split("-")[-1].split(".htm")[-2]
        if year == "europe" :
            year = 2021
        df["Year"] = [year]*df.shape[0]
        return df

In [4]:
tables = [get_table_of_players(i) for i in web]

In [5]:
tables[1].head()

,Pl.,Joueur,Equipe,Buts,dont champ.,Matchs,Min/But(*),Year
0,1.,R. LEWANDOWSKI,Bayern Munich,55 (7 p.),34,47 (117 b/m),75',2020
1,2.,C. IMMOBILE,Lazio Rome,39 (15 p.),36,44 (089 b/m),95',2020
2,3.,CRISTIANO RONALDO,Juventus Turin,37 (14 p.),31,46 (080 b/m),110',2020
3,4.,T. WERNER,RB Leipzig,34 (5 p.),28,45 (076 b/m),106',2020
4,.,R. LUKAKU,Inter Milan,34 (8 p.),23,51 (067 b/m),123',2020


In [ ]:
from powerbiclient import Report, models
    